In [1]:
# @title Imports
from transformers import AutoTokenizer,GPT2LMHeadModel,DataCollatorWithPadding
from torch.utils.data import Dataset,DataLoader
from torch.optim import Adam
from tqdm import tqdm
import torch
import torch.nn as nn

In [2]:
# @title Global Variables
check_point="openai-community/gpt2"
Max_words_count=10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size=8
epochs=2

In [3]:
#@title Loading model and tokenizer
tokenizer=AutoTokenizer.from_pretrained(check_point)

# GPT2LMHeadModel is the gpt-2 model with text generation head(a Linear layer ->   (lm_head): Linear(in_features=768, out_features=50257, bias=False) ) simple gpt-2 model from check_point won't have this head
# it is like a wrapper of gpt-2 model for fine tunning normal gpt-2 model(next word generation) we have to prepare dataset like in input output pair create a loss fn it do all that things
model=GPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
demo_input=tokenizer("This model is downloaded from huggingface.This is a gpt model",return_tensors='pt')
demo_input

{'input_ids': tensor([[ 1212,  2746,   318, 15680,   422, 46292,  2550,    13,  1212,   318,
           257,   308,   457,  2746]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [5]:
tokenizer.decode(demo_input['input_ids'][0])

'This model is downloaded from huggingface.This is a gpt model'

In [6]:
last_hidden_state=model(**demo_input)
last_hidden_state.logits.shape

torch.Size([1, 14, 50257])

In [7]:
tokenizer.eos_token

'<|endoftext|>'

In [8]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adjust_missing_and_unexpected_keys',
 '_apply',
 '_assisted_decoding',
 '_auto_class',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_beam_search',
 '_beam_search_has_unfinished_sequences',
 '_buffers',
 '_cache_dependant_input_preparation',
 '_cache_dependant_input_preparation_exporting',
 '_call_impl',
 '_can_compile_fullgraph',
 '_can_record_outputs',
 '_can_set_attn_implementation',
 '_check_and_adjust_attn_implementation',
 '_check_early_stop_heuristic',
 '_checkpoi

In [ ]:
# @title Dataset
data="""There is no mind. Everything will be, it will be as it is. Nothing is disturbed,
because no one can disturb it. Things will be in their suchness and a little[?]. So the question
is not where to seek truth, the question is not where is truth. No, truth is everywhere, that
which is is known as truth. But the mind we seek [microphone disturbance] if it creates
illusions, if it imposes something on that which is then the illusion is created. So how to
disturb an illusion-creating mind? Now how to attain a mind which does not disturb, which
sees things as they are? So how to achieve a mirror-like mind? That is the question: a
mirror-like mind which has nothing to project, nothing to impose, which just reflects.
So the question, the basic question is never where is truth? how to find it? No, rather ask
subjectively: what is this mind that creates the untrue; why does the mind create untruths; is
there any way to destroy this mind, to become a no-mind? If you can become a no-mind, then
truth is. And truth has been there always. It is there, it has never been disturbed, it has never
changed. When we were not, it was; when we are not, it will still be. It has been there, it is
there. It is the THEREness, but our mind imposes something over it. We see something over
and against that which is. So the untrue is created.
The untrue is a creation of our mind, and if this mind remains the same, you go on being
in trouble. You will be the same, so you will create the same illusion again. A person can
change everything but if the mind remains the same, nothing is changed because the same
mind will create the same difficulties. If he is a householder, he changes, he becomes a
renunciate, but the mind remains the same. And now this new change will create the same
difficulties, because the same machine, the same deception-creating mind is with you. So the
basic problem is not to change stages, not to change outward things, but to change the inner
mind. And there are two types of mind; only two types of mind. A mind which creates
illusion and a mind which does not, but simply waits to see what is. The first type of mind is
an active mind, a creative mind, a creator of deception. The second type is a passive mind; it
just looks, it sees -- it doesn't create. Just like a mirror, it reflects.
This is what I call a meditative mind: a mirror-like thing. And really to call it mind is
nonsense, because it has become no-mind. "Mind" means that it creates mental images. I'm
seeing you, there are two ways to see you: one is to see you as you are: the second is to
project something and see that projection in you. The first will be the truth. The second will
be the untruth. But our mind is such... We are so conditioned in such a way that we never see
what is; we always project something. Somebody comes to me, he never comes with an
empty mind, a passive mind. He has not come to see me, he has come with preconceptions.
And he will be seeing me through those preconceptions. I will not be there; he will see
something that is his own creation. That is the untruth. The conceptual mind is the untrue
mind. And the untrue mind cannot conceive the truth. You understand?
Q. BUT WHEN YOU FUNCTION, AS IN THIS WAY, AND YOU ARE WITH PEOPLE
WHO ARE TRAVELLING, MOVING, HOW CAN YOU MAINTAIN, OR HOW CAN YOU
EVEN ACHIEVE, NIRVANA?
A: No, no! There is no question of maintenance! You have to maintain something if it is
false. If it is, you don't maintain it. You go on moving, you breathe, you never maintain the
breathing. How do you maintain your breathing? Talking, discussing, going and coming, the
breathing is. You never become attentive to it. You are never aware of it; it is just going on.
You are not maintaining it. [A lot of microphone disturbance; next sentence not really
'reliable'] When a state is true, so you don't have to maintain it. If a state moves it will
maintain itself.
If it has not been achieved, then you have to maintain it, but how can you maintain a
thing which you have not achieved? You have to maintain something which is a deception. If
I become a sober man and sobriety is not my state, then I have to maintain it. Then it is a
false deception, an acting. You have to maintain it, Because inwardly you are quite the
contrary. Inwardly you are laughing, and outwardly you are serious. So the seriousness has to
be maintained. But if you are serious, then even when you laugh, the seriousness will be
there; the laughing itself becomes serious. Then you are not that laughing. Then you have
laughed with a totally serious mind and that laughter itself has become serious. Acting has to
be maintained.
And you really asked an essential question, because in our lives we are maintaining
everything. Every type of face needs maintenance, because (LAUGHING) the face is not
real. The face is from the outside, you are something from within: deep double role. This
double personality will need maintenance. and the maintenance will create conflict. This
conflict deteriorates your whole mind.
Then you cannot be spontaneous, then you cannot be real, yourself. You are at every
moment something else, something else, something else. This will kill you! This will become
a burden. And that is why the whole of life has become a burden. Life is just a joy. But the
whole becomes a burden because you have to maintain so many faces. In the morning you are
one person, by the evening someone else. In one moment you are one person, in the second
moment you are someone else. To me you are someone, to somebody else, you are someone
else. Thousands of faces! -- and the maintenance, and the conflict. This becomes
burdensome; this is the burden. This is the sorrow, this is the suffering.
If you are really yourself, you need not maintain anything because you are yourself. Then
you go on, you live... there is no burden and living has become just play, hmm? What they
call in India, leela. Then it becomes leela. Then it becomes play; you do not need to maintain
it.
Maintenance is not the problem. How to attain it? Once attained, attained forever. You
cannot lose it then. There are so many mysteries in life, and one of the mystery is that you
cannot lose that which you have attained. You cannot lose it. You can never lose it; there is
no possibility. It is impossible. What you lose, lose; but you have not attained it. It was a
false faith.
Nothing real can be gained and lost. It is gained and then it is.... Why, why this mystery?
Because the real is really not an attainment; rather, it is a discovery. An attainment can be
lost, but a discovery cannot be lost. It is something that was with you, without you knowing it
-- now you know it. It was with you when you did not know it; it is with you when you know
it. How can you lose it? No real state is ever lost. And no real state ever needs any
maintenance on your part. It is maintained intrinsically, by swabhav: by your inner nature it is
maintained.
And never try to maintain anything. Really just be a floating man, a floating mind. Don't
maintain it. Just be a floating mind. Let everything come that comes, let everything go that
goes. Never cling to it, never welcome it, never try to catch it; let it come and let it go; you be
just at ease. The darkness comes: welcome. It goes: welcome. The light comes: welcome. It
goes: welcome. Misery comes, happiness comes; everything comes and goes, you just be a
witness and watch. And let them come and go. This letgo will make you attain to something
that is not an attainment, but a discovery.
Truth is a discovery. It is neither invention... because invention can never be true.
Invention means the untrue: you have invented it. The family is an invention; the husband is
an invention; the wife is an invention. It is not there, you have invented it. All our
relationships are an invention. Your wife becomes such a bondage; such a bondage because
invented by man. Anything that you have not invented... Love is not an invention, but to be a
husband is an invention. To be a lover is discovery; you have discovered something which
has been there always.
So go on discovering yourself, what you are. Not in a cave... but in moment-to-moment
communication, in moment-to-moment relationship: the inter-relationship. Go on seeing the
trembling, watching...
Q. OSHO, HOW CAN YOU DO THIS WITHOUT DISCRIMINATING? BETWEEN LOVE
AND MARRIAGE... DISCRIMINATING THE TWO.
A: No, no! You are not to discriminate. You are just to search for that which you have not
invented, that which has come to you. Love comes to you, and if society changes, family
cannot be. It is possible. A society can be without family, a society can be without husband
and wife relationship. Mm?You know hippies? and other.... A society can be without any
family. Without any familiar relationship, but a society cannot be without love. Animals are
with love, there is no family, but love is there. So love is something that comes by nature, is
instinctive. Family is something that comes through intellect.
So, one has to discover in his mind relationships, in his movement through life what is
real that comes, or what is unreal that is imposed. If this search goes on, in every walk, in
every aspect, then something -- a mind which witnesses -- will be discovered. Because
through this discrimination; this is real, this is unreal.... Don't ask any scripture, don't ask any
guru. Because if through asking, you cling to some theory, then you will begin to impose.
Then you will say, "This is unreal because Geeta says so." It has become nonsense. You have
lost the path.
Geeta says so, let it say. What you understand, a very little understanding, is so much
valuable than so much great knowledge. Understanding is needed not knowledge. And this
understanding comes through day to day movement; go on discovering, go on seeking what
you are imposing, what you are finding. This discrimination will create a new force within
you. The understanding mind -- that will be created, that will come in force, that will become
a living entity.
That mind liberates. That mind by and by becomes a mirror. Then you neither
discriminate, neither choose. There is no choice then. The mind knows and acts, and there is
no gap between the knowing and the action. And there should not be any gap. The gap, the
gulf, between knowing and the action is the most basic difficulty for a seeker. But that comes
by and by. And in everything: the most little, the most small, the most trivial; there too, you
have to be aware, and seeking and seeking.
Some friend comes to you, you greet him, then let it be known completely to you whether
the greeting was authentic, real or just an imposition, just an acting. You laugh, somebody
comes, says something and you laugh. Discover it within you whether the laugh is authentic,
or just you are laughing to please him, or just as a social gesture. Are you really laughing?
And you will know by and by: this laugh has been false, this laugh has been real; and the real
laugh is a joy.
Real weeping too is a joy because it is authentic. So go on discovering as if is a long
journey. This is so long and so much patience is needed and no tricks and no small... no
shortcut. Through suffering, through seeking a mind comes to a state where everything
becomes a clarity; the reason is attained. You will not rech some God. In the end there is no
god waiting for you. In the end you will reach to yourself, but to a mind which is clear, a
mind which is understanding, a mind which knows. A mind which is passively alert, not
actively illusion-creative, but if someone asks, "what is truth?" then the whole search is
diverted to a pseudo question.
If you ask what is truth, then you will be led into philosophy which is the most nonsense
effort human mind can do. But if you ask, why the untruth? Why I come to untrue? Why I am
not coming to the truth? Then you are asked something else. Another search begins, that
search is religion.
Religion concentrates itself to the mind, to the subjectivity. Philosophy asks objective
questions: What is truth? What is God? Go on asking, go on answering and you will remain
the same because you never touch the mind. You accept it. The mind has been accepted. You
have not changed it, you are not trying to transform it. With the same mind you go on asking
and receiving answers and collecting answers so you can become a scholar. But not a
knower. Then you know so many answers, but not the answer. You know so many things, but
not the thing.
I want that you must understand a very dangerous path leads through this question, "what
is truth?" A very dangerous and very useless, endless and meaningless quest, because the real
question is not what is truth? The real question is what are you? Who is on the path to seek?
If he is pseudo, false; if he is an actor, if he is not authentic, then go on searching. An
unauthentic mind, how it can reach to the truth?
Truth comes to a true a mind. It comes by itself to a true mind. You can know only that
which you are; you cannot know that which you are not. If you are untrue, untruth will be
coming; if you true, truth is bound to come. The same is attracted to the same. An untrue
mind repels the true; it seeks but it repels. So if an untrue can collect definitions,
philosophies, systems, but that is not the real seeking.
Turn it! Never ask what is true, ask how the mind becomes such that the true comes to it.
You see the distinction? You understand the distinction? The question is not, where is the
sun?; the question is, how to open the window so the sun may come in. It is there, it has been
always there. If your window is closed, it will be there. But the window is closed; so sun is
not, the darkness is. And someone asks in the darkness, what is the light? What is the sun?
Where to seek it? Where to go? And goes on circling, circling that very dark room, because
your mind is the dark room. So you cannot go outside it. You will have to break some
windows, some doors in this mind. If you go on with this mind through lives, you will be
asking the same: what is light? What is light? And the mind is dark and there is no window,
there is no opening, there is no door, and goes on asking. And in the darkness, it consoles
itself by collecting some answers. Geeta, Ramayan, Quran, Bible it collects.... And in
darkness it is consolated that I am knowing, knowing, knowing, by and by I will come to
know; I have known so much answers. But the mind is dark, and there is no opening.
So I say throw your scriptures and throw you rules. And break your mind so that the light
may come in; it is ever-ready, ever-waiting. The moment the window is open, the light
comes. And the whole dark corner is changed, something else has come in and then you can
never lose it.
So concentrate on your mind, not any objective truth. Make your search concentrated on
your mind; the mind is the problem. And then there is no philosophy; and there comes
religion. Religion is existential, experimental. Philosophy is speculative: what is truth?...
[Tape break here: change in sound, background noise, and His voice indicates an entirely
new setting.]
... Then you become totally independent. It is independence one needs. Outwardly will be
gained in any occupation. But an occupation like gardening or farming will do much good.
Bodily, mentally and spiritually -- on all layers of life, it will be helpful. Because you will be
independent, totally independent, and when you are occupied in some work if you maintain
simultaniously an awareness... while doing, acting, working; simultaneously there must run a
current of awareness within you. Then the occupation itself becomes an object of meditation.
[Tape side B]
Rather to choose some artificial object of awareness, it is better to choose some type of
occupation which is in both ways helpful; helpful as an object for meditation, and helpful as
to your maintenance, economic maintenance and independence. It will do much good. If you
go somewhere to some farm, and to remain there working for three or four hours per day, and
then for twenty hours you are completely free... To remain unoccupied for twenty-four hours,
necessarily means to be dependant on someone, to some institution or to some person, or to
live on some charity. But that charity or that institution or some person automatically, they
create a sort of bondage....
QUESTIONER: OBLIGATION.
Osho: Obligation. An obligation is so heavy that it necessarily creates a situation where
independence is lost and it's particular type of anxiety is created. And that anxiety is so much
inner that it obstructs the inner current of awareness. So it is double. Better to go to some
farm or some garden, but it is better if you choose farming or gardening than any other
occupation. Because other occupations are so much involved; and either (way) you will be
mechanical or other types of routine work. And routine work which is the same every day
creates boredom. And that boredom too becomes an obstruction in the inner current of
awareness. But an occupation like farming -- it is so living, it is so new every day, it is not a
work, rather it is a pleasure: to be growing plants and all that atmosphere that surrounds it,
that natural atmosphere. All that helps so much that it is better to be occupied.
Q: ALSO COULD I ASK YOU ABOUT MEDITATION PRACTICE BY... I CANNOT
UNDERSTAND [THE WAY IN WHICH] IT SHOULD HELP.
A: Hm mm. To be meditation means an effort to be aware. To be aware of the whole
process of life, but in the beginning one can chose some object like breathing. It is a very
natural object to be aware: The ingoing and the outgoing of the breath. Be aware constantly:
the breath has gone out, it has come in; and the rhythm and the process and the in and out
going phenomena, be aware of it. The moment you will be aware, you feel a certain change, a
certain chemical change within your mind. Because when you become aware of the breathing
process, mind becomes silent. The silence is automatically created. Because your total mind
is engaged in seeing, in witnessing the breathing. And the rhythmic circle of breath creates an
inner music. That inner music too leads you to more and more silent and sweeter moments
within. When you are walking or bathing or eating or doing anything, doing farming, be
aware of your breath. But this awareness, this effort of awareness should not be a strain; it
should be effortless. The very effort should be effortless. It should not be strenuous, it should
not become a tension, you should not do it upto an anxiety is created or a strain is created or
you feel some inner tension in doing it. The effort should be effortless. It means do it without
any strain.
Q: IF YOU CANNOT KEEP IT UP, THEN YOU JUST WAIT UNTIL THE MOMENT
ARRIVES AND YOU CAN KEEP... I FIND WHEN I'M TRYING TO PRACTICE
AWARENESS, IT DOES NOT LAST VERY LONG.
A: Mm, It will not last in the begining. It will not last for very long in the beginning, but
go on doing it. Whenever you become conscious that you are not doing it, do it. If you forget
it, let it be forgotten, again when you remember it, do it. Do not repent that the current was
lost. Oh no, there is no repentance. Whenever you become conscious, four or five times in the
day when you become conscious, do it. When you forget it, let it be forgotten. Again you will
become conscious, and this becoming conscious and doing it without any repentance, without
any strain just leave when you will remember it. And how much time you remember it,
doesn't matter. Even for some seconds or some minutes, if you remember it, that is enough.
By and by the stretch will become longer, by and by you will remember more, by and by the
gaps between two remembrances will be lesser. And in a certain period -- it depends how you
do it -- in a certain period, you will remember it constantly. But don't long for it. Just be
thankful, if you remember only for few seconds, just be thankful. That is enough. Enough for
today. Again tomorrow the stretch will become longer, gap will be shorter and the intensity
will go deeper and in a certain period, you will remember it without any strain. It will go on
simultaneously with your all work, with your all activities. You will be doing everything
outside, and there will be the remembrance within. You will be aware. That awareness will
continue.
That continuity comes by itself if you just go on doing it. But never repent for it. That it
was not as much longer as I longed for. There is no question of longing. As much as it comes,
be thankful for it. It is enough.
This will do, this will create that effortless effort. Effortless effort. Whenever you
remember, for a single moment, be aware and let it go. Don't cling to it. Don't forcibly do it.
Force cannot be helpful. It should be spontaneous without any effort, without any strain.
Even for a single moment, if it comes in a day, even for a single moment, it will become a
seed within you and the seed will go on growing and growing and growing. And someday
you will find that the remembrance has become part and parcel of your conscious mind. Then
in the longer run, even in the night when you are asleep, sometimes, some moment will come
that you will remember even in the sleep and you will become aware. Then the continuity
will penetrate in the sleep and then a circle will be created. But that requires time and
patience and a loving gratefulness. Don't be repentful for those moments when you were not
aware, but be grateful for those moments when you were aware. Emphasize those living
moments when you are aware, don't think about those moments which have gone without any
awareness.
It will be created; it is not difficult. When it is created, one knows how foolish one was; it
was so simple, it was so easy, it was so natural. But when it is created, then one knows.
Before that it seems so difficult, but the difficulty is not with the process. The difficulty lies
with our impatience. We are so impatient, we ask for everything without any preperation. We
ask just here and now. Even... it can come just here and now, but that needs infinite patience.
If you have infinite patience, then this very moment can become a moment of explosion. But
our whole mind, our whole make-up of the mind is based on impatience. That impatience
creates an inner barrier. Then, meditation should not become a part and parcel of our desire.
Because you cannot be meditative with a desiring mind. Because a desiring mind creates
another current of desire against the current of awareness. Desire is part and parcel of
unconscious mind. So if you desire too much, even if you desire for God, even if you desire
for liberation, even if you desire for meditation, samadhi, this too will become a barrier. So
don't desire. Just be curious to know, to know what is meditation. Don't make meditation to a
means towards anything else. Meditation should be the end. We want to know the complete
total silent mind. Why? Because we have known the [1 word unquick] mind, we have known
the non-silent mind, we have known the mind with desire, we have known the mind with a
longing and a clinging, and we have suffered for it. The suffering has been so much. We have
known this and the suffering. Now we want to know the opposite mind and also to know
what comes with it.
We do not know so we cannot desire, we do not know what will be the outcome of a
silent mind. But we have known the non-silent mind, we have known the suffering that
comes with it, the darkness, the dukhha, but we do not know what pleasure, what bliss will
come with the mind when it becomes meditative, aware and silent. But we want just to know
the opposite. We have suffered this much long, for so many lives. We want to know what is
the other mind. That other mind is too our mind, but it has remained inactive because of the
activity of this mind. So this mind becomes silent, that mind it is deeper, inner, much more
inner, will explode and a new type of vision, a new type of life will come out of it.
But don't desire it. The desire... Because every type of desire is part and parcel of this
mind which we want to transcend. So if you desire, this mind goes on working again. And
this mind goes on creating patterns of suffering again. Let this mind go. Because we have
suffered. Let this mind go because we are suffering. And let the new mind come, with no
desire, with no particular longing, let it come. And see what happens.
Much happens. FIRST QUESTIONER: WHAT IS YOUR MISSION IN LIFE? AND WHAT METHOD
SHOULD WE USE TO FULFILL THIS MISSION?
OSHO: As far as my mission in life is concerned, there is no such thing as my mission.
Neither is there any my, nor is there any mission. I am not teaching something to others --
there is no message to be given to the world. Messages are many, missions are enough -- and
there is no lack of missionaries, nor is there any lack of thoughts, ideologies, isms. On the
contrary, the mind of the world is much too burdened with these things. To me it seems that
if the mind of man can be unburdened, only then is there any possibility of living the truth, or
of feeling life in its totality. Ideologies, thoughts, isms, missions, they all add to the burden of
the mind.
The more the mind knows, the less it becomes capable of knowing; knowledge is the only
hindrance towards knowing. Knowing is something quite different from knowledge.
Knowledge means thoughts, and knowledge means borrowed learning. Knowing means a
mind which is unburdened of everything that is known; a mind in the state of knowing is
simply ignorant -- it doesn't know anything, it is humble. And to be humble, one has to
unlearn what has become a burden.
The known must cease for the unknown to be. And life is unknown, and truth is
unknown, yet we are all burdened with knowledge. This attitude of knowledge becomes a
hindrance towards the void, which goes into the unchartered, into the unknown, the
unacquainted, the unlearnt.
So to me there is nothing to be preached or to be taught -- I am not a teacher in that sense.
Rather, I am awakened -- not a teacher. And the first awakening that is required today is the
awakening of the humble attitude of an unlearnt, unknowledgeable mind, a mind that is open
and not closed.
This humility is needed but you cannot teach this humility through a mission, because a
mission becomes an organization. A mission becomes, in the long run, the vested interest. A
mission becomes a sect; a mission becomes interested, not in the unknown, but in the
knowledge that the mission has to impart to others.
So every type of mission burdens the human mind. And the need is to be unburdened.
There are things which can be known by others; information can be imparted. As far as
science is concerned -- the complete knowledge known as scientific knowledge -- there are
things that can be imparted. Then there is the possibility of a school, of a teacher, of a
mission.... But there are things as far as the inner is concerned... as far as the divine is
concerned, and these things cannot be made part and parcel of a dead knowledge. They
cannot be condensed into maxims, nor is there any possibility for any objective
experimentation or for a laboratory where more than one person can experiment and come to
a conclusion. There is no such possibility. The inner, the divine, is basically individual, is
basically subjective. One knows, but cannot impart the knowledge to others. One knows and
lives.... Others may feel the perfume, may feel the scent, may feel the song, may feel the
unknown presence, but that too is intuitive, that too is indirect.
You cannot preach the divine directly, so there is no mission-like activity for me. When
there is no mission, there is no question of how to work it out, how to implement it, how to
organize it. I am against all sorts of organizations. Truth cannot be organized, and the
moment one thinks of organizing it the truth is killed, only the dead remains. What is living
goes out of any organization, because to know the truth is so individual, it cannot be
organized. You can organize around an ideology, not around a realization. You can organize
around a mission, not around the unknown, not around the realization of the unknown.
So there is no possibility of there being any organization around me. Nor am I interested...
rather, I am against it. But we can never think of doing anything without organization. And
the difficulty is: the very nature of the thing is so, that if you organize it, you kill it; and if
you don't organize it, then you raise the problem, what to do with it? How to make others
know about it? That too can only be done through individuals, not through organizations.
If what I say appeals to you, or somebody, he has to become the embodied presence of
what I am saying. Even if one individual becomes the embodied presence and people begin to
feel about him... something new, something transformed is imparted. By the very presence of
such an individual the work goes on....
Only such a type of mission can I conceive of. But I cannot call it a mission because the
very word has become associated with so many wrong things.
To be religious means to be imparting indirectly what one has realized. So I will go on
trying, endeavoring to contact individuals personally, and indicating towards things which
cannot be indicated. But no dead code or dead scripture must be allowed to be practiced or
organized, or given a base -- unlike the old sects, the old religious or public organizations.
To me, organization means something based on hatred. No organization is centred around
love, because love needs no organization. Hatred needs organization. Nations, parties,
religions, are all based on hatred: hatred of others, hatred of those who are not in our fold, of
those who are not under our flag, of those who are not convinced of our message. Hatred is
such a great force, it organizes. It is so poisonous, and a single individual is not able to be so
poisonous, a single individual cannot be as dangerous as hatred requires one to be. So one
needs organizations, others, a crowd -- one cannot be alone.
But love is such a silent force, that to be alone is enough. Love doesn't know how to
work, it works itself. There is no question of method -- I have never been concerned with
method.
I am a man with no method -- absolutely no plan, no method, no organization. My only
hope is this, that love works spontaneously. It needs no method. It needs no previous
rehearsal. It works! And if it works it is alright, and if it doesn't work then too it is alright. If
love is not enough, then no organization can be much more.
And love has never been methodological. Hatred has always been methodological. When
You hate, you hate with a plan, you hate with a disciplined mind. You have to be conscious
of the enemy, of the fear, of the planning of the opposite party. You cannot be spontaneous.
With an enemy you are always to be prepared. You cannot be unprepared. But when there is
no question of hatred, when there is no one to be thought of as an enemy, when there is only
love to work and to live with, there is no question of tomorrow, there is no question of the
future -- it is spontaneous, it works. And the very spontaneity is its force. If you don't plan
your hatred, you will be defeated; but if you plan your love, you will also be defeated.
So, I have no plans for the future -- today is enough. This very moment passing, the
present moment is enough. What I am doing, I am doing without any method. If it is
something, if it is to be of any use for others, it will spread like fire without any method.
SECOND QUESTIONER: WHEN YOU SAY THAT KNOWLEDGE IS A HINDRANCE
TO THE AWAKENING OF MAN, THEN WHAT TYPE OF KNOWLEDGE ARE YOU
AWAKENING IN HIM? DO YOU WANT MAN TO BE ILLITERATE?
A: No, not illiterate. I am not saying that man should be illiterate, I am saying that man
should always be ready: open, not closed. The knowledge that one has gained through the
past must not become a hindrance to the future.
SECOND QUESTIONER: THAT IS KNOWLEDGE ABOUT RELIGIOUS THINGS
ONLY?
A: Of course, of course. When I say knowledge, I always mean religious, because, really
there is no such thing as knowledge which is not religious. Scientific knowledge only means
scientific information. There is no scientific knowledge. It is only information -- so, it can be
imparted. There is no such thing as scientific knowledge.
SECOND QUESTIONER: BUT IS THERE KNOWLEDGE ALSO IMPARTED BY
SOMEBODY?
A: No. If it is imparted then it is not religious. Neither is it religious, nor is it knowledge.
One has to know oneself. When it comes through others, it becomes dead. Then it becomes
simply information, it is not religion. "Religious" means that which has been experienced,
that which is existential, that which has been got through living not through scriptures, not
through teachers -- not borrowed, but lived.
Religious knowledge is only knowledge. Why do I say so? Because science can know
something about things. When science knows something, it knows something about it. It is
always about and about; it is always an acquaintance from the outside. A scientist knowing a
flower... he knows something about it! Everything that can be known about it, he knows. He
tries to know more and more, but it always remains information. He has not gone into it, he
has not become one with the flower. He has not known it from the inside, he has known it
from the outside. Knowing from the outside is what I mean by information. Knowing from
inside is what I mean by knowledge. And man can know only himself from the inside;
everything else will be known from the outside. So only religious knowledge, knowledge
which means a knowing of oneself, knowing the true inner reality... such knowledge only is
true knowledge, because it knows, it is not information about oneself. But we can gain and
cultivate information about ourselves. Then again it becomes non-religious. So, scriptures are
not religious. They may be scientific or not scientific, but no scripture can be religious
because it can only give information: it can give what Mahavira knows, what Buddha knows.
I can know what Buddha knows, but that is not my knowledge.
SECOND QUESTIONER: HOW IS THAT A HINDRANCE?
It is a hindrance because when someone comes to conceive that he knows and really he
does not know -- he has only been collecting information and this collection of inner
information gives him a conception of knowledge -- then he won't try for further knowledge,
then he won't try for himself. Then it will become a hindrance, because he has come to
conclude that he knows, and really he doesn't know. This misconception of knowing will be
the hindrance. And further, when the mind is enclosed in information - and there is much
information around the mind, much knowledge, much scripture -- a barrier is erected between
oneself and that which is. When you come to a flower, if you don't know anything about it,
you have to contemplate it. But if you know something about it, then you pass it by, because
the barrier of knowledge... you know, I know about it: this is a rose, this is a beautiful flower,
you say, and you pass on. There has been no living contact between that which is called the
rose and the mind which is in a wrongly conceived enclosure of knowledge. There is no
living contact. The rose can be known, not through any knowledge about the rose, but
through the direct contact with its living existence. But still a rose is something outside. You
cannot even think about yourSELF, because who is to think? And every type of information
is a part of your thinking. It makes your mind a thinking machine. You go on thinking and
thinking, and reading and reading, and you go around and around a concept. But if one is to
know oneself, one has to cease to think. Because, the very thinking leads you afar.
SECOND QUESTIONER: THEN WE SHOULD NOT READ BOOKS ABOUT...?
No, I am not saying that. I am not saying that you should not read, neither am I saying
that the information is of no use. I am not saying that. I am saying this: you must read, you
must know, but you must know also that this knowing is simply information, and not
knowledge for you. If you are aware of it, then it can't become a hindrance, then your mind is
always open. You are always ready to learn, you are always ready to discover. Then you have
not arrived, you are still searching, discovering. But a man of knowledge comes to think of
himself, that he has arrived. A pundit, thinks of himself as a man who has arrived. Then there
is no travel, then there is no mutation, then there is no transformation -- then there is no
further search; the mind has become closed with knowledge. This may seem contradictory,
but this is the truth: a mind that thinks it knows becomes incapable of knowing.
THIRD QUESTIONER: IT IS SAID, "AS YOU THINK, SO YOU BECOME." IF ONE
THINKS OF THE REAL SELF....?
No! This is one of the very wrong conceptions. This conception says, that as you think so
you become. No, you cannot become as you think. But you can feel that you have become
something. As you think, you can feel that you have become something, you can imagine it.
But imagination is not knowing. If someone thinks I am brahma, I am god, and goes on
thinking it, and thinking and thinking, then he comes to feel that he is divine. But this feeling
is not knowing. This is simply imagination being suggested, repeated, hypnotized. He has
hypnotized himself by suggestions, by constant repetitions.
You cannot become what you think. Rather, the contrary is true: what you become you
can think about. The becoming is first, the thinking is secondary -- because the being which
you are is always behind the primary foundation of your thinking.
First you can exist without your thinking, but your thinking cannot exist without you. So
you are foundational, the thinking is secondary. Thinking is only on the periphery, the
circumference; your being is in the center. So one can be without any thoughts, but no
thought can be without any being. If you think something and imagine something, you may
come to a feeling -- imaginary, psuedo -- but you cannot come to your very being, because
that which you don't know... how can you think about it? We can only think about something
which we already know. If you know your self, then there is no question of any thinking
about yourself. If you don't know your self, then how can you think about it? Then you will
borrow something from others....
SECOND QUESTIONER: KNOWING ONESELF WILL BE THINKING IT.
No! Knowing one self will be a total cessation of thinking; it will be simply existing in
oneself -- not thinking. Thinking is always about something. Thinking cannot exist in a
vacuum, it is always about something. You can think about a chair, you can think about a
house, you can think about a friend or an enemy; you cannot think about yourself, because
you are! Who is to think? So, if you are to know yourself, you have to cease completely --
totally -- the process of thinking....
The process of thinking must not be there then you are, in your simple authentic
existence. Then you are, simply. Then you come to know that mind is not through thinking,
that mind is through non-thinking. That's why I define meditation as a state of mind which is
not thinking, but is aware -- not thinking and aware. There is no thinking in the mind, but the
mind is, and is totally aware. So, about what can it be aware now? There is nothing to think,
so it cannot go outside....
SECOND QUESTIONER: AWARENESS... IS IT ALWAYS SOMETHING IN CONTACT
WITH OTHERS?
A: No, it is not. You can be simply aware. There is no need of...
SECOND QUESTIONER: AWARE OF WHAT?
A: Aware of yourself. You are aware of yourself. But these two things are one -- you and
your awareness are not two things. When you are aware of your house, there are two things.
But when you are aware of yourself, there is nothing which is aware, and nothing to be aware
about -- you are the awareness. Then you are simply awareness. Then you are not aware, you
are awareness, because there is only one thing: you and you and you, and awareness -- and
nothing to be aware about. This moment, when you are not aware but you are awareness, is
meditation. That awareness which is not aware of anything else; it simply exists, like light; it
simply exists, like a flame.
SECOND QUESTIONER: WOULD IT BE JUST LIKE OBJECTIVE....?
A: No. When awareness becomes objective, when you become aware of something else,
then it becomes thinking. Then thinking goes on, thinking is created.
FIRST QUESTIONER: WHEN YOU COME SUBJECTIVELY?
A: When you come subjectively, then there is no possibility of any thinking.
THIRD QUESTIONER: HOW DO YOU DEFINE, ATMA CHINTAN?
A: There is no such thing as self-thinking, atma chintan. You cannot be in a state when
you are thinking about yourself, because one who knows, knows; he never thinks about it.
We only think about things which we don't know... you go on thinking. But one can know
oneself through borrowed knowledge; then, one can think about oneself. Then one can think
about atma and the nature of the self, the existence of the self, the origin of the self... one can
go on thinking about it. But this is not knowing.
In meditation, you are not thinking about yourself, you are knowing. And the moment of
knowing is the moment of explosion. The mind is exploded -- you become something else.
You are not the old man, you are not the old person. The old person has gone, a new man has
come into being. Now this man lives in a totally different way, exists in a totally different
way, loves in a different way, is in a different way because he has come to know himself. The
first point of knowledge has been achieved. Now he can know others too....
We are in a dilemma. We don't know ourselves, and we think that we know others. We
are trying to know others without knowing ourselves. This is impossible. If I am ignorant of
my own self, how can I know anything else in the world? The very center is ignorant, the
very person is in darkness, so how can I discover light anywhere else? -- it has not been
discovered in me!
The first point of discovery, the first explosion of light, must be, can only be in me. Then
it spreads, then it goes out and out and out. This first moment of transformation is
self-knowing. Then like a pebble thrown in a lake, the circles go on spreading and spreading.
Then they will go on up to the bank. The explosion is in the self, but then the explosion goes
on encircling the whole world. It goes to the very end, and then it becomes divine knowledge.
First the explosion is self-knowledge, then the explosion goes, goes, goes to the infinite
and becomes divine knowledge. Self-knowledge is the door, is the opening towards the
divine. But we do not know ourselves, and we think we know even God. We debate about it,
we struggle about it, we discuss it, we confute, refute, convince... we say this is right, this is
wrong, this religion is right, that religion is wrong, this scripture is accurate, that scripture is
not accurate, we go on debating and discussing things for which, in ourselves there is no
opening.
The opening comes through the explosion of the self.
One thing more to be understood: The explosion is the explosion of your ego also. When
you know, when you come to know yourself -- this is a miracle and this is a mystery -- the
self remains only in ignorance. The self is part of the ignorance. We say self-knowledge, but
really when the knowing comes, the self goes. There is no self then. Then there is only
knowing. Then you are not. Then there is only existence, then there is only being. That being
is atman.
Atman is not the self, the ego. One thing to be understood: You cannot know yourself,
because the moment you come to know, you are not there, only the knowing remains. And
that knowing without an ego, goes to the very endless end. Because there is no boundary
now.
THIRD QUESTIONER: THEN, THE WORDS ATMA SAKSHATKAR HAVE NO
MEANING FOR YOU?
A: Really, all these words are conceived dualistically. In fact, there is nothing like atma
sakshatkar, "realization of the self," because when the realization is, self is not, and when the
self is, realization is not. So "realization of the self"... these are two contradictory terms.
It is like this: If there is darkness in the room, there is darkness. Then you bring a lamp to
know darkness. You bring a lamp because, in the darkness, how one can know what darkness
is? So you go out and you come back with a lamp to know the darkness. But when there is
the lamp, there is no darkness! Yet when there was no lamp, there was darkness. So there is
no enlightening of the darkness. There is no such thing as darkness seen in the light. There is
no such thing.
There is no such thing as self realized. Self is ignorance; when you come to realize that
which is, there is no self.
Buddha was more accurate in his expressions. He said, "realization of no-self." That is
exactly nearer to the reality than realization of the self -- realization of no-self, an
enlightening of no-darkness, because when the realization comes, there is no self. So Buddha
says that no self is the reality. That could not be understood; it is very difficult. He said:
existence of no self, anatman. There is no atman, there is anatman. And he meant something,
something meaningful. But he could not be understood. We can understand atman because it
approximates the ego. We can think about the self because it is something like an extended
ego, a bigger ego.... So we can conceive of it....
THIRD QUESTIONER: BUDDHA SAYS "NO ATMAN"...
A: Buddha says no atman.
THIRD QUESTIONER: MAHAVIRA SAYS THERE IS ATMAN.
A: Mahavira says there is atman, but without the ego: there is no ego -- atman is. Buddha
says there is no atman. And the reality is they mean the same thing.
But I say, Buddha's expression more approximates the reality, yet Mahavira's expression
is more likely to be understood by all. The expression is different; but they mean the same
thing. Mahavira says, there is self but no ego: When you realize the no-ego, you will know
the self. Buddha says, there is no self. When you realize this, you will know existence, you
will know the being, you will come to know.... And if you ask Buddha what we will come to
know, what will be realized, he'd say, realize and know, because when you ask what, you
have again asked a dualistic question. You are again asking aware of what? So Buddha says
come to awareness, and then you will know there is awareness but you will not know
anything about it.
That awareness is freedom.
THIRD QUESTIONER: UP TILL NOW WE HAVE HEARD YOU ON MAHAVIRA AND
BUDDHA, OR ON THE ATMAN. CAN YOU ENLIGHTEN US ABOUT CHRIST'S OF THE
ATMAN? HAS HE GIVEN ANY LIGHT ON THIS?
A: Not in such a way as to be compared with Buddha or Mahavira. Jesus speaks a totally
different language. Jesus' language is totally different, he talks about the Kingdom of God.
THIRD QUESTIONER: HE ALSO CAME TO A STATE OF ENLIGHTENMENT
HIMSELF?
A: He came, he came... he understands. He realized what any Buddha or any Mahavira
realized. But the language is completely different. The language is not Indian.
First, the language is not philosophical. Second, the language is Jewish, conceived in a
Jewish frame, with parables, with metaphors, with stories. And if one is going to understand
Jesus' territory, then one has to imbibe the spirit of Jewish thinking. They don't talk about the
self, they talk about the kingdom of God. They say: we are as we are, we don't know about
the kingdom of God. Whatever the kingdom of God means, we do not know anything about
it. We are unneccessarily poor, we are unneccessarily beggars. In the innermost parts of
ourselves there is a kingdom and if we go in, the kingdom will be revealed, and we will then
be children of God -- not beggars, not crippled with desire, not crippled with misery, we will
be bliss. This kingdom of God is the same as what Mahavira calls atman, Buddha calls
no-atman. This is the same, but Jesus speaks in parables: he calls it the kingdom of God.
SECOND QUESTIONER: WHAT ABOUT KRISHNA?
A: The same. Those who have realized....
SECOND QUESTIONER: WHAT DOES HE SAY ABOUT THIS?
A: He says the same thing. He calls it brahma: when you surrender yourself.... Krishna
says surrender yourself to me. To me means to the ultimate, surrender yourself to the
ultimate. Surrender yourself to the ultimate and you will become one with it. Because what
one knows, becomes what he knows. But the question is of surrendering. Then it's the same.
Mahavira says, egolessness. But how can you be egoless without surrendering? These are two
things taken from two different points of view but coming to the same place. To be egoless
means to be surrendered, completely surrendered. A person who has completely surrendered
himself becomes egoless. And when one becomes egoless, one realizes the truth, because the
ego is not there to be a hindrance, to be a covering. Krishna emphasises surrendering,
samarpan, sarva dharman parapechii, mamekam sharanam, prarchi(?): leave all, and come
to me totally surrendered. But this me means the ultimate, this me is not Krishna.
THIRD QUESTIONER: THE ULTIMATE MEANS....?
A: God, the Divine, the real, the truth, the atman, whatsoever one may name it -- Krishna
emphasises surrendering, Buddha emphasises no-self, Mahavira emphasises egolessness,
Jesus, the kingdom of God, the bliss. He talks about the bliss, ananda, the kingdom... and the
poverty in which we live....
SECOND QUESTIONER: WHAT WAS....?
A: Ananda -- happiness. Happiness unbounded, happiness unlimited, happiness infinite.
Whosoever may have realized, whosoever may have reached, the question is not that one
says this and the other says that -- this and that mean the same thing. If one has realized and
the other has not realized, then even this and that cannot mean the same.
This is to be understood: A person who has realized, he may say whatsoever he likes. He
cannot say against it... anyone who has realized. But a person who has not realized may say,
"this is what I mean, this is what you are saying, I am totally convinced of it." But he cannot
mean the same, because the same has not been experienced.
Words cannot convey the experience. But anti-words can convey.... If the person is a
person of knowing, anti-words can convey. Mahavira says, "Self is the only thing. To know
the Self is the aim, to realize the Self is all." Buddha says quite the contrary: "To know the
self is ignorance."
These two statements quite contradictory, quite opposite to one another, mean the same
thing. But if someone says, God is, and another, too, says God is, and if the persons
concerned are persons who do not know what God means, these two statements -- similiar,
exactly similiar: GOD IS -- these two statements cannot mean the same. The two persons and
their experience and their ignorance and their definitions, are different. They have not come
to the point where individuality dissolves.
Unless individuality is dissolved, we cannot come to the same experience, because the
difference is not in the experience, it is in our personality, our individuality: I. My I gives
meaning to my words, your I gives meaning to your words, and your I and my I are two
different things, they can't mean the same thing. But a Buddha has no I, a Mahavira has no I,
a Jesus has no I. So they cannot mean different things, because the difference was created by
the I. They can give different expressions, but they cannot mean different things. For us, this
becomes a dilemma. This has become a very difficult job for the human mind to conceive,
that Jesus, Buddha, Mahavira, Mohammed, Krishna, they all mean the same thing. Then...
then what is the foundation of Hinduism, of Mohammedism, of Jainism, of Christianity?
In reality, there is no foundation. These all are conceived in ignorance. We have not
understood what Jesus means, therefore there is Christianity. We have not understood what
Mahavira means, therefore there is Jainism. These are misunderstandings -- words with
different meanings that we have conceived as if the experiences were different. So there are
different sects.
To me, there is no difference, the difference is only of name. One is Krishna: if you wash
off the name... one is Mahavira, and if you wash off the name, and the two persons were here,
nameless, you could not see the difference. Where is the difference? There is no difference.
But the name for us means much. Krishna, Jesus, Buddha, the names are more important to
us than the reality for which those names stood.
If one goes deeper, one comes to know the source is one. These things may differ, the
paths may differ, but the reaching point is the same. Those who were realized come to know
the one reality. Those who were not realized, conceive their own realities individually.
We create our own realities, our own philosphies.
Another question has been asked,
WHAT IS YOUR PHILOSOPHY?
I have got no philosophy at all. Because philosophy means how I define reality.
Philosophy means how I systematize the real, what I say about reality.
The first thing I say is that the reality cannot be said, cannot be expressed. How do I
define it? I never define it, because definitions can only be in words. You cannot define love.
You can know it, you can feel it, you can live it, you can suffer it, but you cannot define it.
How can you define love? How can you define prayer? You can be in prayer, there are
moments when you are in prayer, but you cannot define it. All that is beautiful, all that is
true, all that is good, cannot be defined. It can only be experienced?
So I have got no philosophy.
SECOND QUESTIONER: SO, WHEN YOU EXPERIENCE SOMETHING THEN YOU
CAN DEFINE IT, CERTAINLY?
A: No! Even then you cannot define it! You can only express it... falteringly. You can't
define it. You can express it falteringly, and a person who knows will always say
categorically... he will say that what I am saying is not what I have experienced. Because
words convey so differently....
It is like this:
You have come to a lake; you have seen the sun rise and you have gone back. Now
someone asks: define the beauty, define the sunrise! What do you mean that you have
experienced a beautiful scene? What do you mean that you have become exhilarated? What
do you mean? Then he says, please paint that sunrise. And you begin to paint. You paint a
sunrise: a sun, -- with a pencil -- a lake, hills, but then you see the difference. It was
something living, but this is something dead. That was something miraculous, this is a sketch.
A circle, and you say this is the sun? A circle drawn with a pencil, you say this is the sun?
SECOND QUESTIONER: THE DRAWING MAY DIFFER, BUT THE EXPRESSION
CAN BE SHARED...?
A: The drawing too is an expression... and it could be more approximate, more real, even
more real than words -- because words are more abstract.
SECOND QUESTIONER: BUT IT CANNOT BE WRONG? THE EXPRESSION...?
A: That is not the question, that is not the question. I am not saying that it is wrong. I am
not saying that it is incorrect, I am saying it is something so faint....
SECOND QUESTIONER: ... EXPRESS IT FALTERINGLY?
A: I said falteringly. A person who has seen the sun rise, may understand something from
your sketch. But a person who has not seen any sunrise, he cannot conceive of what you
mean. This circle with a pencil, this is the sun? This is beautiful, you say? How nonsensical!
How can this circle with a pencil be beautiful?
It can convey something only if the person concerned has experienced it -- then there is
no question, then it is symbolic, then it can be conveyed. But then there is no need! A person
who has known the sunrise, he will understand when you say you have seen a beautiful
sunrise. He will say okay, alright, he won't ask you to define it: "Draw a sketch, because I
have not seen it, so I may understand by your definitions...." But if you have seen it and then
you draw it, you know something has gone wrong. Something has gone wrong. It is
something so faint, that it would have been better not to draw it, because it will convey
something which you are not meaning to convey. You could say this is a sketch, but no
comparison can be made. Even I myself cannot say this is beautiful. Something else was
there -- living; this is something dead.
And a sun is a physical object. There is a possibility that you may not be able to draw it,
but you can have a photograph -- that would be more, real. You can have a color photograph,
then it would be more real. Still it is an objective experience.
But a person who comes from the divine, cannot bring a photograph with him, cannot
have a color photograph. He comes from a vacuum. When he faces us, he is in one of the
most difficult states. He knows something, he has known it, and now you are confronting
him, encountering him, asking him, and he is at a loss. He is nowhere. He cannot concieve of
how to say it....
So there have been many realized persons for whom the very impossiblity of defining the
experience has led them to be silent. They have remained silent.
THIRD QUESTIONER: JESUS HAS SAID THAT THOSE WHO HAVE EARS, LET
THEM HEAR. WHAT DOES HE MEAN BY THAT?
A: He means something. He means something which... he means that only having ears
doesn't mean that you can hear. Only having eyes doesn't mean that you can see, because
there are things which cannot be seen by the eyes. There are things which are seen with
closed eyes, there are things which are heard with closed ears.
A person who has heard things not heard by ears comes to you, says something, and says
this: if you have ears...? You will say, what nonsense you are talking about? I have got ears, I
have got eyes... But he says I have seen something which was not seen by eyes, but I have
seen it, and seen more exactly than any eye can see. If you have eyes -- those eyes which are
not these eyes, those ears which are not these ears.... So Jesus says, "Listen, hearken -- but
only those who have ears, who have eyes, can see."
There are eyes which can be opened. But if you think these are all, that these eyes are the
only implements, instruments of perfection, then you are losing a greater world -- because
there are things unseen by these eyes, which can be seen, which can be heard, which can be
felt, which can be known. That's what he says. It is the same thing.
SECOND QUESTIONER: THAT REALIZATION STATE?.... HOW TO AVOID
THINKING WHILE REMAINING IN THE WORLD, AND KNOWING YOUR OWN SELF?
A: No, you cannot avoid. You cannot escape, because what you avoid follows you, and
what you escape from, in a very deep sense you become involved with.
So don't escape, don't avoid. Thinking is, be aware of it. For some moments, sit quietly,
be aware of the thinking, be aware of the process: thoughts are coming, thoughts are going...
like breath coming and going, coming and going constantly -- there is a continuity. See it;
just see it. Stand aside, be aware of it. Thoughts are going... this thought has come, that has
gone, see it, know it. Notice it, but don't condemn, don't say this is good, don't say this is bad.
Don't choose. Just be aware. Like a person standing on a street: people are going and coming,
the street is running, there is a crowd, traffic, and a person is standing, is just watching. So
just see the process.
By and by, the more you become alert, awake, the more you begin to see thoughts, the
more you will come to feel intervals. A thought will come, then it has gone and another
thought has not yet come... there is an interval. There is a gap -- a little gap. But in that gap
there is no thought. So the more alert you become, you will feel the intervals.
Those intervals will be first glimpses of the self -- first glimpses. But glimpses. Just
glimpses come and gone.
But go on! Those glimpses will be so blissful, that they will invite you more and more
towards the inner. And the more you become alert, conscious, thoughts will be coming less
and less -- it is in proportion. The less alert, the more thinking; the more alert, the less
thinking. The proportion goes like this: the more you become conscious, thoughts come less
and less. For some time the road becomes vacant and you dip in, you go in. If you continue
this -- this awareness, being a witness to your mind -- then there comes a time... no one can
predict it -- it is unpredictable but it comes.... Then some day you feel that there are no
thoughts.
The thought process has completely ceased and you are transformed to somewhere else,
somewhere you have never been. A door has opened, you have gone in. You are not even
conscious of going in -- you have gone. You know the change, the transformation, the
transcendence -- you have transcended. Then you come back, but you are a totally different
person. Then you live in this very life, but quite differently. Then your everything has
changed. It has become more intimate, more loving, more compassionate, you have become a
loving, a truthful, a silent, a peaceful, an aware mind. And everything that was associated
with ignorance, conflict, tension, all have gone. Something has been achieved which has led
you towards the source. You have become one with it. Then you live blissfully in every
moment, in every circumstance, in every condition, in every situation, something silent and
blissful accompanies you like a shadow.
That is the bliss. It is that we have been longing -- for births. That has been the longing,
but we have searched for it somewhere else, where it is not. We have been longing for it... in
love we have been longing for it, in richness we have been longing for it, in everything that
we desire, we have been longing for it. But the search has gone astray, not in the right
direction. It has gone outward. And the more we have gone outward, the less has become the
possibility of the bliss for which we are searching, for which we are desiring.
SECOND QUESTIONER: THEN, THE MAJORITY OF PEOPLE ARE WASTING THEIR
TIME IN WORLDLY AFFAIRS, DOING THIS AND DOING THAT... GOING TO THE
MOON?
A: Those things may be meaningful in another way, in another sense. But as far as the
bliss is concerned, they are all useless. They are irrelevant -- not relevant to the basic search,
the basic urge, the basic thirst of humanity. That is somewhere inside. And you come inside
when thought ceases. With thought, the known ceases, the knowledge ceases. With thought
the wandering ceases. With thought, everything outward ceases -- thought is the foundation
of all that is outward. You come in -- really, you have never been out, but with the
association of thought you have imagined yourself to be out. When I say you come in, you
really don't come in, you have always been in, but for the first time you are disassociated
from the outside. You realize what has been always yours. A: The mind is now integrated. The moment you are one the will is created. The will
means an integrated mind. The willessness comes from disintegration, from fragmentariness,
from your mind divided in itself; divided in contradictions, struggling, conflicting in itself.
The mind which is unconscious is a willless mind. The mind which is total, one and whole is
a mind with a will. The will only means wholeness, total.
FIRST INTERVIEWER: DOES THAT MEAN CONCENTRATION?
A: No, it doesn't mean concentration. Concentration means still. It's a fragmentary
attitude. When you are concentrating, you are choosing a particular point excluding all else.
Q: IS THAT WHOLE (ALL?) ?
A: Not whole(?). The will means you are whole within, as well as without. It is a
choiceless wholeness. You are not exclusively concentrating, but you are whole. The moment
you are whole within, the total without is open to you, you are open to it. It is non-exclusive,
it is inclusive of all. When I concentrate upon something, all else is excluded. When I am just
whole in me nothing is excluded; everything is included. The whole totalness within, as well
as the totalness without is will. To concentrate means to exclude.
FIRST MAN: (INAUDIBLE) AT ONE POINT...
A: Yes, if you are concentrating to one point your whole mind power, there will be two
things; First you cannot concentrate on a single point your totalness. Your totalness can only
be a response towards the totality. If the without is taken as a whole, only then you can be
whole within. You cannot narrow yourself to a point, your whole mind can only be whole
when the whole world is taken as one. Concentration is not only excluding something
without, it is narrowing yourself also. It is focusing your mind to a particular narrow passage,
ultimately to a point.
Q: (SOMETHING LIKE THIS) THEN THE MIND MUST BE...
A: Narrowed
Q: ... NARROWED, DON'T YOU THINK.
A: Yes
Q: SO IT MUST BE STRONG.
A: It can be strong, it can be strong, but it cannot be whole. And a strong mind is always
hiding (one word inaudible), but a whole mind is not strong in that sense. The whole mind is
whole, it has no weakness. It is as it is. If you are concentrating, a strength will be there, a
particular type of strength that comes from narrowing, that comes from focusing, that comes
concentration will be there, but behind it there will be weakness. Against that weakness you
have created this strength. You as a total being will not be strengthened. A particular
moment, a particular exclusive point, a particular fragment of your consciousness will be
strengthened against the weakness of your whole mind. So this strength cannot be a
conscious strength. It will come and it will go. And the moment it goes you will feel even
more weaker. The moment following concentration will be still a more weak moment. You
will feel exhausted. And you will feel something has gone out of you. Because this
concentration is not of the total. It is only part of your mind which has been narrowed. And
this effort of narrowing, exhausts. When I say will, to me will means wholeness. We must be
whole within. It is not concentration on something, but it is just to (one word) your
fragmentariness. And you be whole only when you respond to the whole, when you are aware
to the whole without, (1 word) the without and the within. Being a total response. Only then,
the will is there. This will is neither strong nor weak because it is not divided. So no
contradictory terminologies can be applied to it. It is natural, it is tao, it is the whole. It is as
strong as possible, as weak as possible, simultaneously. It is just like water; humble, weak,
and strong. And aggressive. It is not like a stone; strong but hiding weakness within. When a
strong stone comes in the way of the humble water passage, -- primarily the stone will seem
to be strong and water weak, but then ultimately there will be no stone. And water will be
flowing (above?) The water is neither weak nor strong; it is both simultaneously. The
strength comes from the wholeness, also the humbleness. So only a strong person, totally
strong can be a strong person. His humbleness is not weakness. The total mind is neither
male nor female. The total mind is both simultaneously. It is as active as male, it is as passive
as female. Both simultaneously. The whole is always both. Because (one word never?) one
chooses against the other. So either you can say weak and strong, both active and passive,
both alive and death. Or you can say it is neither. It transcends both. The will to be as I will
be is a wholeness. It is not a desire, it is not a concentration, it is a not a choice at all. The
whole from within responding to the whole without. And the moment the whole responds,
there is no within and no without. It is one, it is both. The within and without. All the
contradictions, all the paradoxes, all that is taken as opposite is dissolved in it. This will can
be said the nature, the tao, the dhamma. And if you are total in this sense, then you are both.
The only (1 word) cause and effect and makes them three. Because that too is one of the
contradictory oppositions. Cause and effect is a bondage, but bondage to one who is fighting
against it. Every type of bondage is built on the fight against it. If I am fighting against my
bondage, then it is bondage; if I am at ease in it, co-operating, one with it, then there is no
bondage. The bondage comes from the antagonism. The law of cause and effect, the
dhamma, is not a bondage for a mind like a Buddha, it is not a bondage. It is bondage for us.
It is bondage for us because we are always standing against it, doing something against it,
feeling something against it. We have not taken it as our nature. If it becomes our nature, it is.
If we understand, can comprehend it, can conceive as our nature, then there is no bondage.
Then it becomes leela, then it becomes playfulness. We have rules and regulations even in a
play. You cannot play without rules and regulations. You cannot play -- even a play has it's
own rules and regulations. But we know that these are to make the play possible, but still it is
a play. Rules are there and they are to be followed. And the more you know they are rules of
a play, the more you become capable of following them because rules of play cannot be
fought. They are to be cooperated. Because they exist. Only when you cooperate with them...
A total mind is neither free nor a slave. He is both. He is a slave with his cooperation, he is a
slave just as in a play you come under rules and regulations in a play. Games have their rules
and the sportsmanship means to be under them. To be freer (clear) and (inaudible) then
seriousness and non seriousness become one. The whole thing is if you think life in it's
totality, then you become the life. If you take life in it's life, then you become the Dhamma,
you become the tao. All that is under cause and effect is not now taken as against it, as
against causality; it becomes the causality. It is not me as an individual who is under cause
and effect; the cause and effect and my own-ness are one and the same. No one is under
anybody else. We are under ourselves. But this realization can come only when the total
(inaudible) We are ourself fragmentary and thats why we conceive the world in a segment
and the total is just a total; mathematical total, arithmetical total. We have taken our existence
in fragments and then arithmetically we have totalled it. This is not the total. This is only
fragments added. But fragments added cannot make a total. Because total is always
something more than a part. For example, we can take a painting. The arithmetical total will
mean this quantity of color, this quantity of canvas, this type of brush, this type of hand, this
type of man painting it. So if we can conclude all this, it will be a arithmetical total, but the
painting will not be there. The painting is something more than the total of its colors, the total
of all its fragments -- it is something more. The whole transcends its parts,. It is not simply an
arithmetical total, a total always transcends the parts. It is more than the part. A machine is a
more than its parts. You can heap its parts, but that will be an arithmetical total, but it will not
work. It will be a dead heap. It will be a total, but a dead heap. But when the machine
becomes an organic whole, it is more than its parts, it is more than its fragments totalled
arithmetically.
So the living total is always more than the fragments. If we conceive the world not as it
exists in its totality, neither we conceive ourselves as we exist in our totality. Our whole
conceptions are arithmetical. I have seen something, I have not seen the whole, even then we
are seeing there is choice. So different persons nurturing different cultures will see different
things. When we are seeing it is a choice. A person comes in this room he will see something
else, another person comes he will see something else because both are choosing. Being is so
great a thing, that I cannot see it, so we choose. And the descriptions will be different. What
we are hearing is a choice; it is not the whole that comes to our ears. It is a choice. Then we
accumulate all these chosen things from our senses, and make a picture of the whole. This we
call the world, this is not the world; this is your world chosen by you a particular individual.
The world is much more. So when I say to be whole, I mean a choiceless existence. You are
not choosing, you just are.
By understanding, by understanding this fragmentary attitude, by understanding this
fragmentary mind, by understanding this divided (inaudible), by understanding and knowing
and becoming aware of the part which you have taken as your mind -- it would be better to
say this is not mind, these are minds -- we are not a mind, we are minds. The Mind must
come as you become more aware of these minds. The more you become aware of your
minds, the Mind is being born. The mind comes in. That awareness will ultimately become
your Mind. These minds will lose by and by their existence, their separateness and they will
dissolve in the awareness, then you will be just aware. Without any mind, you will be a Mind.
We are with mind, that's why we cannot be a Mind with a capital M. We are always of a
particular mind. When you see, you see through a mind, when you listen, you listen through a
mind. If you can become aware of these minds, these fragments, these conditionings then a
mindless Mind begins to arise in you. Then a awareness unconditioned by your past -- these
fragments are conditionings from your past. When you become aware of these conditionings,
then a Mind which is not of the past, which is to arise, which is to pulsate(?)...
SECOND WOMAN : (INAUDIBLE)
A: Yes. The moment it becomes whole, then there is no difference between the individual
and the cosmic. The individual and the cosmic is the difference created by our mind. (Tiny
tape break) ... then there is no difference, the difference falls. It is not that the individual mind
becomes cosmic, no, we come to know that there is no difference between the individual and
the cosmic. It has always been the one. Only the fragmentary mind, minds that we have
created through experience, through education, through cultivation, through the path; those
fragmentary minds, those crystallizations of past experiences were the dividing barrier. Now
when they are not, the division has fallen. It is not that the individual now becomes the
cosmic, and neither that the cosmic becomes the individual, no, there is no individual and no
cosmic now. It is one and the same. It has always been so. We have lost something which we
ourselves have created. The reality has always been so. It has never been known a division.
The division was in mental division; creation of our own conditionings. The reality has been
always undivided, indivisible. It is still so, it has always been so, it will be so. But we,
through our cultivated mind see it as divided. The moment you become whole, the moment
you become aware then there is neither individual nor cosmic. Or you can say, this is the
cosmic. It makes no difference, but cosmic is not against the individual; cosmic as dissolving
all the individual. How this can be achieved? By being aware of your fragmentary mind, of
your conception, of your attitude, of your approaches. If you can become aware of your mind
which sees, which hears, which chooses, then you are seeing it; a flower -- it is not only that
the flower is there, between you and the flower there is a particular mind, a particular attitude
about the flower -- that is still between you and the flower. Otherwise there is no barrier.
Otherwise you and the flower are two extremes of one existence, two ends of one single
moment. The consciousness and the flowering are two extreme points of one process. If your
particular mind is not there, if it is there and it is always there, it is not that you are just seeing
the flower, your seeing has a method, your seeing has a conception, your seeing has a like or
dislike, your seeing has a path. You say, "This a rose." Now there is no rose and the rose
itself has never known that it is a rose. It is we who have called it a rose. The moment you
say rose, all that is associated with the word rose stands between you and the flower. And
much is associated. If your culture says that a rose is beautiful, then the flower becomes
beautiful. Your culture is against the rose, then it becomes ugly. If your experience is... of
this flowering is associated with pain, with thorns, then the rose becomes different. If your
associations are not associated with pain but with play, then the rose becomes different. And
the rose is all the time same. You come with a mind to it and your mind destroys that which
is real and creates an imaginary, hallucinatory thing. If you have no mind, if you can see this
rose with no mind and the seeing is totally innocent, completely refreshed. If the seeing is not
something from the past. If you and the rose are in this moment living simultaneously, with
no mind on your part -- because there is no mind on roses' part. If you are without mind in
this moment, just aware, just existential, then the rose is not known as something separate.
Then it is known something as part and parcel of your consciousness. Then the flowering of
the rose is your flowering; then the perfume of the rose is your perfume. And your
consciousness is roses' consciousness through you. It is rose himself known himself through
you; via your consciousness rose has come to know himself. Then the feeling is like this. It
cannot be expressed as (inaudible) But either that you have flowered and the rose... or that the
rose has become conscious through you about himself. The feeling is like that.
Q: (INAUDIBLE) IS ALSO SEEING?
A: This is seeing really, this is seeing. This is darshan -- with no mind.
Q: (INAUDIBLE) THIS KIND OF SEEING?
A: I don't know, what he says. I say this is seeing. Only when you are mindless the seeing
is there, the knowing is there, the feeling is there. The mind is a destroyer. The mind is a
destructive force. Don't try to be whole. You cannot try for it. There can be no effort.
Because any type of a effort, is a effort of a particular mind against other minds. That's why
the effortlessness of it is to be understood clearly. You cannot achieve it, because every type
of achievement is a longing of a particular mind. You can only understand. This is so, this is
suchness. The mind is fragmentary and the mind is not one, it is poly-psychic. It is minds.
What are these minds? These minds are the experiences of the past with which you have
become attached, associated; with which you have created a clinging. Why you have created
the clinging? Because to exist mindlessly is dangerous, to exist mindlessly is insecure, to
exist mindlessly is to be always in the unknown. That's why all the has become known to us
we have made it a part of our consciousness. All that has become an experience, knowledge
we have been clinging to it. Against the unknown, against the insecure, against that which is
coming, we are clinging with that which has gone. These are our safety measures. So a mind
which is longing for security cannot be mindless. This is to be understood. Nothing is to be
done. This is just a fact. This is to be understood that a mind which is longing for security can
never be mindless, because a mind longing for security will cling with past experiences, past
knowledge, past information and will create mind. And will always be (inaudible) both dead
minds. And you cannot be alive through dead things.
Reasons. First that we have not a single mind, we have minds.
Second, these minds are our past experiences.
Third, we are clinging to these minds, because of the fear of the unknown, because of the
possibility, infinite possibility of all that is future, of all that is coming. The past is something
(inaudible) because it has happened, it is dead. You can do something with it. You cannot do
anything with the future. Once you become aware, that these are these bindings; longing for
security, this safety with the past, this deadness, this (inaudible) of the past, you can do
something with it and the unknown too And the unchartered future. And the infinite
possibilities. About which you can never be certain. If you can understand these two things:
the certainty of the past and the uncertainty of the future, and if you can understand that life
means uncertainty, life means insecurity, life means to be in danger -- only a dead person is
out of danger. Now he can not become dizzy, now he cannot die. There is no death for him
now, he is at ease. A dead person is always at ease. Everything has become certain.
Everything has happened and now nothing is to happen, so there is no danger. But if you
understand that life means uncertainty, life means insecurity, life means dangerously, life
means the unchartered unknown possibilities. If you understand this, then by and by, minds
will drop. By and by they will cease to function, by and by you will become one and whole.
And by and by the response will be total. This total response is religiousness. Total response
every moment, in every situation. Be total within and dig the total without. The moment this
happens, the totalness within and the totalness without they become one. Then there is no
barrier. There can never be two perceptions. The moment two perceptions come close, they
become one. The wholeness is always one. So they say, there is a saying in the Upanishads,
'If we take the whole out of the whole, the whole remains behind. Nothing is taken out.
Because you cannot take anything from the whole. Even if you take the whole, the whole
remains behind. You cannot add anything to the whole, if you add the whole, it remains the
same. The whole means the one. And there can be no methods to it. All the methods are for
the understanding. All the methods are to create a situation, in which you can understand
yourself, your fragmentariness, your poly-psychicness, your multiplicity...
SECOND MAN: THIS IS ALSO SEEING
A: Yes.
Q: DIFFERENT KIND OF SEEING, BUT WE ARE SEEING.(INAUDIBLE) THAT IS
REALITY
A: Yes.
Q: AND THIS KNOWING OF MINE IS ALSO SEEING?
A: It is also seeing. Not different. It is the same. When you know yourself in its totalness,
only then you can know that is without in its totalness.
The first thing to be seen is yourself. Because if you have not seen yourself then no
seeing can happen without.
Q: SOMETIMES AFTER THIS MEDITATION METHOD I CAN SEE MYSELF FOR...
A: Yes, you can see. You can see.
Q: SOMETIMES.
A: Sometimes it will come over you. Sometimes when you are whole, the whole will
explode over you. And this will go on becoming more natural. It will go on becoming by and
by... the gaps will be lesser, the intervals will be less. And then a moment comes, a moment
of explosion from which there is no return.
Q: (INAUDIBLE) THIS IS JUST AN OLD WORD, BUT THEY CALL IT WILL OR
WHAT?
A: These are all words. They can be used, but the will is the power, is the energy. There is
nothing such as will power. Will is the power, will is the energy, will is everything, but by
will I mean the wholeness. And those who use the word will power, use it as in a psychic
thing. They use it as a power concentration. I am not using that word in that sense. To me
will is the whole. The only God is the will. We are never the will, we are desires. Even if a
desire is concentrated, it is a desire power. The will is the power. With no desire, it is power
with no movement, or a inner movement. It is a power with no direction, or directionless,
dimensionless. When we say God is power it doesn't mean God is powerful, it means God is
equal power. It is the power. So when I say will is power it is not that will has a power -- will
is power, will is energy. It is like energy. And you can only attain to it, when you have lost
yourself. Until you are there, desire will be there and the power will only be a desire power,
fragmentary, created by concentration, by narrowing it, by exclusion. This power cannot be
your life. You will have moments of powerlessness also. This potency is always against your
impotency. This will come and go. But when the will comes, now nothing is to go,
everything is. Even the flexibility is powerful. Even the impotence becomes a potency. Thats
why, I would like to say the word 'Tao' is better than the word 'will,' than the word 'dharma,'
than the word 'law,' tao is better. Because tao is both; the (inaudible) the (inaudible), the
darkness the light, the potency and the impotency; it is both.
SIDE TWO (starts after 1 min 33 secs empty tape)
... only then power becomes absolute. But will power as ordinarily used, is only power
created by conflict, power created by concentration; that is not power. This is just creating a
conflict in you and making your one mind stand against the other. And because of this
encountering a certain energy, a certain force is created -- you can use it, but there will be
moments following it of depression, of powerlessness. But when there is no direction, when
there is no desire you are power. Undirected to any goal. You are energy, you are life. Then
there is no shadow. Then you exist without shadow. Then you exist without the opposite
following (inaudible) Thats why it is said, "God is shadowless." He is, but there is no shadow
to him.
(nice chuckles)
[Next question in Hindi. The English translation is omitted here as much of the Hindi was
inaudible]
3RD MAN: (INAUDIBLE)
A: It is a very symbolic experience. The moment you transcend the body the extension
comes. The body is the only limitation. It is not that the tree is other, it is not that the flower
is the other, it is only the body that makes them look as other. The moment you become
bodyless, the moment you feel that you are not body, then everything is one, because now
there is no barrier-barrier to the extension, to the extending consciousness. That is the reality,
this is the falsehood. You looking me as the other, I looking you as the other this is the
falsehood. Falsehood created because of the identification of myself with my body. If the
identification drops, then there is no barrier, then you are me, then I am you. That is a very
strange phenomenon, to mate for the first time, it is the most strange phenomenon.
3RD MAN: NOW I REALLY UNDERSTAND WHAT( ?)
A: Yes.
3RD MAN: I THINK I UNDERSTAND WHAT IT MEANS NOW. (OSHO LAUGHS) I
(INAUDIBLE) (?)
4TH MAN: (INAUDIBLE) BEFORE THAT JUST BETWEEN ZAZEN IS OFFERED FOR
SUDDEN ENLIGHTENMENT, BUT ZAZEN IS SOTTO(?) SOTTO IS GRADUAL, CHI IS
SUDDEN AND SOTTO IS GRADUAL, BUT IF YOU SAY ZAZEN IS SOTTO, IT'S FOR
SUDDEN ENLIGHTENMENT.
A: Yes. But in sotto they use the word zazen, but they are not just sitting. Only the word
is there. They are doing something. That is what we were talking. They are doing something,
they are meditating, they are contemplating, they are concentrating, they are not just sitting;
so only the word is of sotto, they look as if they are just sitting, but they are all the time
engaged in doing something. The word zazen means not doing anything just sitting. Sitting
only.
Really there is no sect and there is not any possibility of there being any sect. Who can
practice zazen? Individuals have been there, a Buddha just sitting. A Bodhidharma just
sitting. Whenever we come to them we go to them we ask what to do? Then they (how
come?) and the just sitting is dropped. And the Buddha is forced to answer us. If we sit just
go and sit, then it is no answer for us because we were asking how? It is irrelevant, we were
asking how to sit and he is saying just sit and the thing will happen. So two (inaudible) will
look we have been sitting all the time, it has not happened. Though we have not sat in that
way, just sitting is the most difficult thing. Whenever we come to a person like a Buddha; his
all these teachings are these teachings of just sitting. But when we to a temple and see a
Buddha we will say he is meditating, he is in a asan, he is in siddhasan, he is in padmasan, he
is meditating, he is in a dhyan... this is all nonsense. He is just sitting, he is not doing
anything, neither... not even dhyan! He is not doing it. He is in dhyan. And in dhyan, in
meditation means not doing meditation. So there has not been any effect, there have been
claims, there have been word, but the moment a sect is created the how comes in. All those
who will come they will ask how to do it and I go on telling them just go and sit. So (more
and more?) come and there will be no sect. Because they ask how and I say there is no how,
just sit as you like, but just sit. It becomes meaningless to them. A sect is created the moment
how is answered. And the moment how is answered, there is no possibility of Zazen. The
Sotto use it, but it is not there. The zen says that you must be effortless, but they make all the
effort. They say the meditation must be effortless, then they develop methods to achieve this
effortlessness, they become(?)) efforts. A pure religious sect is an impossibility. Only absurd
religious sect is a possibility. A religious sect is to be in existence, must be absurd in
contradictory. He cannot be consistent. If the religion is pure then it evaporates, you cannot
catch it. If it becomes impure you catch it, then it is not religion.
4TH MAN: IF CHI(?) AND ZEN ARE NOT TWO, BUT ONE...
A: No, no they are all not two, the difference is only of the how. The difference is always
of the how, and how is basically absurd. The achievement, the realization is always in
effortlessness. The knowing comes only in the moment when the doing is not. The being
reveals when the doing has ceased. So this is the purest concept, this is the purest thing
possible to say, but when I say to you, you will ask how to be in such a position, how to be in
such a situation. Then I must, I am compelled to speak in contradiction. I will go on telling
you that just sitting is the only real thing, and at the same time, showing you methods which
are not of just sitting. But doing these methods a moment comes when you yourself become
aware of the absurdity; what you are doing? You cannot go to the being through doing. The
doing must cease. But through doing you can become aware that this is all absurd. The
moment meditation becomes absurd, you realize meditation. The moment there is no need of
any dhyan, the samadhi is achieved.
Early Talks
Chapter #4
Chapter title: None
22 September 1970 pm in CCI Chambers, Bombay, India
Archive code: 7009220
 ShortTitle: EARLY04
 Audio: Yes
 Video: No
[NOTE: This is a partly edited tape transcript of an unpublish."""

In [10]:
data=[i for i in data.split(".") if len(i)>2]
data[0]

'There is no mind'

In [11]:
def tokenize_function(example):
    return tokenizer(example, truncation=True,return_tensors='pt')

tokenizer.pad_token = tokenizer.eos_token
def custom_collator(batch):
    input_ids = [item["input_ids"] for item in batch]
    attention_masks = [item["attention_mask"] for item in batch]

    # Pad sequences manually
    input_ids_padded = torch.nn.utils.rnn.pad_sequence(
        input_ids, batch_first=True, padding_value=tokenizer.pad_token_id
    )

    attention_masks_padded = torch.nn.utils.rnn.pad_sequence(
        attention_masks, batch_first=True, padding_value=0
    )

    # Labels = input_ids for causal LM
    labels = input_ids_padded.clone()

    return {
        "input_ids": input_ids_padded,
        "attention_mask": attention_masks_padded,
        "labels": labels
    }

In [29]:
tokenize_function(data[0]).values()

ValuesView({'input_ids': tensor([[1858,  318,  645, 2000]]), 'attention_mask': tensor([[1, 1, 1, 1]])})

In [13]:
class customDataset(Dataset):
  def __init__(self, data):
      self.data = data
  def __len__(self):
      return len(self.data)
  def __getitem__(self, idx):
      tokenized=tokenize_function(self.data[idx])
      return {"input_ids":tokenized['input_ids'][0],"attention_mask":tokenized['attention_mask'][0],"input_ids":tokenized['input_ids'][0]}

In [14]:
train_data=customDataset(data[:int(len(data)*0.8)])
test_data=customDataset(data[int(len(data)*0.8):])

train_data_loader=DataLoader(train_data,collate_fn=custom_collator,batch_size=batch_size)
test_data_loader=DataLoader(test_data,collate_fn=custom_collator,batch_size=batch_size)

In [15]:
next(iter(train_data_loader))

{'input_ids': tensor([[ 1858,   318,   645,  2000, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256],
         [11391,   481,   307,    11,   340,   481,   307,   355,   340,   318,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50

In [18]:
#@ title optimizer
optimizer=Adam(model.parameters(),lr=1e-3)
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [19]:
# @title Training
for i in range(epochs):
  tot_loss=0
  for g in tqdm(train_data_loader):
    x=g['input_ids']
    attn=g['attention_mask']
    labels=g['labels']
    x=x.to(device)
    attn=attn.to(device)
    labels=labels.to(device)
    optimizer.zero_grad()
    loss=model(input_ids=x,attention_mask=attn,labels=labels).loss
    loss.backward()
    optimizer.step()
  print(f"Epoch {i} Loss {loss.item()}")

100%|██████████| 127/127 [00:19<00:00,  6.51it/s]


Epoch 0 Loss 1.1705496311187744


100%|██████████| 127/127 [00:19<00:00,  6.43it/s]


Epoch 1 Loss 1.069006085395813


In [22]:
tokenizer(tokenizer.eos_token)

{'input_ids': [50256], 'attention_mask': [1]}

In [34]:
# @title inference
def inference(input_sent):
  pred_tokens=[]
  last_token=-1
  while last_token!=50256:
   input_sent_tokenized=tokenize_function(input_sent)
   input_sent_tokenized=input_sent_tokenized.to(device)
   output=model(**input_sent_tokenized)
   last_token=output.logits[0][-1].argmax().item()
   pred_tokens.append(last_token)
   input_sent=input_sent+tokenizer.decode(last_token)

  return tokenizer.decode(pred_tokens[:-1])

In [39]:
inference("Hii.")

'\nQ: (INAUDIBLE) (inaudible) then you become one with it'